<a href="https://colab.research.google.com/github/fmunin/RAG-Chatbot-PowerBI/blob/main/frm_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#IMPORTING KEYS
import os
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [4]:

from google.colab import drive
drive.mount('/gdrive')
urlBase="/gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot"

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [5]:
# Installing the required libraries
!pip install openai==1.66.3 tiktoken==0.9.0 pypdf==5.4.0 langchain==0.3.20 langchain-community==0.3.19 langchain-chroma==0.2.2 langchain-openai==0.3.9 chromadb==0.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/

In [1]:
!unzip /gdrive/MyDrive/classwork/Generative-AI/RAG-Chatbot/PowerBI.zip


Archive:  /gdrive/MyDrive/classwork/Generative-AI/RAG-Chatbot/PowerBI.zip
  inflating: Introducing_Power_BI.pdf  


In [6]:
# Importing the standard Libraries
import time                           # For measuring execution time or adding delays
from datetime import datetime         # For handling timestamps and datetime operations

# ChromaDB Vector Database
import chromadb  # Chroma: a local-first vector database for storing and querying document embeddings

# OpenAI SDK
from openai import OpenAI             # Official OpenAI Python SDK (v1.x) for interacting with models like GPT-4

# LangChain Utilities
# RecursiveCharacterTextSplitter intelligently breaks long text into smaller chunks with some overlap, preserving context.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Loads all PDF files from a directory and extracts text from each.
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Base class representing a document in LangChain; useful for downstream chaining and processing.
from langchain_core.documents import Document

# Embeddings and Vector Store
# Generates vector embeddings using OpenAI’s embedding models (e.g., `text-embedding-3-small`)
from langchain_openai import OpenAIEmbeddings

# Integration for using Chroma as the vector store within LangChain’s ecosystem
from langchain_chroma import Chroma

# Chunking Source PDF File


In [7]:
#Set up directory where PDF SOURCE files

urlpdfSource = os.path.join(urlBase,"source-docs")
print(os.path.exists(urlpdfSource))
print(urlpdfSource)

True
/gdrive/My Drive/classwork/Generative-AI/RAG-Chatbot/source-docs


In [8]:
#text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=16
)

In [28]:
# Chunking the source pdf
loader = PyPDFDirectoryLoader(urlpdfSource)
chunks_PowerBI = loader.load_and_split(text_splitter=text_splitter)


In [29]:
# Inspecting content of first page
print(f"The text splitter create {len(chunks_PowerBI)} chunks of text")
print('======= Printing the first record ==========')
chunks_PowerBI[0].page_content

The text splitter create 407 chunks of text
======= Printing the first record ==========


'Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo'

In [16]:
# initialize Chroma DB collection name
collection_PowerBI ="PowerBI-Documentation"

# Initialize OpenAI Embedding Model

In [18]:
#defining API key
openai_api_key = userdata.get('OPENAI_API_KEY')

In [19]:
# Instantiate the OpenAI embedding model
embedding_model = OpenAIEmbeddings(
    api_key=openai_api_key,
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='text-embedding-3-small'
)

In [20]:
# Initialize a persistent Chroma client
chromadb_client = chromadb.PersistentClient(
    path="./PowerBi_db"
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [21]:
# Pinging the database client to check if the connection is alive
# the heartbeat method returns the current time in nanoseconds and is generally used to check if the server is alive
chromadb_client.heartbeat()

1768425638402375502

In [22]:
# Confirm database is empty
chromadb_client.count_collections()

0

In [23]:
# Instantiate a Chroma vector store to store and retrieve document embeddings
vectorstore = Chroma(
    collection_name= collection_PowerBI,
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_model,
    client=chromadb_client,
    persist_directory="./PowerBI_db"
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [27]:
# confirm collection
chromadb_client.count_collections()


1

In [26]:
# Confirm database has been populated with the collection
chromadb_client.list_collections()

['PowerBI-Documentation']

In [30]:
# Batch 500 chunks to send to the API at a time, pausing execution for 30 seconds afterward
i = 0 # Initialize the starting index for the chunks

while i < len(chunks_PowerBI): # Iterate while the index is less than the total number of chunks
    vectorstore.add_documents( # Add documents to the vector store in batches of 500
        documents=chunks_PowerBI[i:i+500], # Get the current batch of 500 chunks
        ids=["text_" + str(i) for i in range(i, i+500)] # Assign unique IDs to each chunk in the batch
    )

    i += 500 # Increment the index by 500 to move to the next batch
    time.sleep(30) # Pause for 30 seconds to avoid rate limiting issues with the vector store


In [32]:
# how many records are in collection??
PowerBI_Collection= chromadb_client.get_collection(collection_PowerBI)
PowerBI_Collection.count()

407

In [33]:
PowerBI_Collection.peek()

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['text_0',
  'text_1',
  'text_2',
  'text_3',
  'text_4',
  'text_5',
  'text_6',
  'text_7',
  'text_8',
  'text_9'],
 'embeddings': array([[-1.41233020e-02, -3.10449544e-02,  3.87464017e-02, ...,
         -7.75077497e-04,  3.25128931e-04,  1.71608292e-02],
        [ 9.72380396e-03,  4.84161591e-03,  2.15032659e-02, ...,
         -1.99479982e-02, -1.87173076e-02,  1.61477346e-02],
        [-1.45157333e-02, -5.66877611e-03,  1.01762936e-02, ...,
         -7.89197534e-03, -4.34708036e-03,  4.59537562e-03],
        ...,
        [ 3.20041440e-02,  1.93384122e-02,  1.15462057e-01, ...,
         -1.53100910e-02, -1.23954285e-03,  1.92148443e-02],
        [ 1.96178108e-02,  2.66210157e-02,  9.81774256e-02, ...,
         -1.15210460e-02, -1.11496222e-04,  2.94929929e-03],
        [-1.55997900e-02,  7.47539988e-03,  5.77180181e-03, ...,
         -1.78922853e-03, -3.84286046e-03,  7.89604150e-03]]),
 'documents': ['Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo',
  